#演示 PydanticOutputParser 的用法

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import PydanticOutputParser,StrOutputParser
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
llm = ChatOllama(
    model="qwen2.5:7b-instruct",
    temperature=0,
)
# llm=ChatGoogleGenerativeAI(model='gemini-2.5-flash-lite',temperature=0)

class Person(BaseModel):
    name: str = Field(description="person's name")
    age: int = Field(description="person's age")
    occupation: str = Field(description="person's job")

class People(BaseModel):
    people: list[Person] = Field(description="list of people")

# 创建解析器
parser = PydanticOutputParser(pydantic_object=People)

# 创建带格式说明的提示
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract information about people from the following text."),
    ("human", "{text}"),
    ("human", "{format_instructions}")
])

# 完整链
chain = (
    {"text": RunnablePassthrough(), "format_instructions": lambda _: parser.get_format_instructions()}
    | prompt
    | llm
    | parser
)

result = chain.invoke("Alice is 30 years old and works as a doctor. Bob is 25 and is an engineer.")
print(result)

#演示StrOutputParser

In [ ]:
def demo_str_output_parser():
    # llm=ChatGoogleGenerativeAI(model='gemini-2.5-flash-lite',temperature=0)
    llm = ChatOllama(
        model="qwen2.5:7b-instruct",
        temperature=0,    )
    """演示 StrOutputParser 的使用"""
    print("=" * 60)
    print("📝 StrOutputParser 演示")
    print("=" * 60)

    # StrOutputParser 是最基础的解析器，直接返回 LLM 的文本输出
    str_parser = StrOutputParser()

    # 基础用法
    prompt = ChatPromptTemplate.from_template("写一首关于{topic}的短诗")

    # 构建链
    chain = prompt | llm | str_parser

    result = chain.invoke({"topic": "春天"})
    print(f"🎨 诗歌结果:\n{result}")
    print()
demo_str_output_parser()

#演示并行处理

In [4]:
# llm = ChatOllama(
#     model="qwen2.5:7b-instruct",
#     temperature=0,
# )
llm=ChatGoogleGenerativeAI(model='gemini-2.5-flash-lite',temperature=0)

from langchain_core.runnables import RunnableParallel
str_parser = StrOutputParser()
# 并行处理多个任务
parallel_chain = RunnableParallel({
    "summary": ChatPromptTemplate.from_template("Summarize: {text}") | llm | str_parser,
    "keywords": ChatPromptTemplate.from_template("Extract keywords from: {text}") | llm | str_parser,
    "sentiment": ChatPromptTemplate.from_template("Analyze sentiment of: {text}") | llm | str_parser
})

results = parallel_chain.invoke({"text": "I love this new AI technology!"})
print(results)

{'summary': 'This is a very short and enthusiastic statement expressing strong positive feelings about a new AI technology.', 'keywords': 'Here are the keywords extracted from "I love this new AI technology!":\n\n*   **AI**\n*   **technology**\n*   **new**\n*   **love**', 'sentiment': 'The sentiment of the sentence "I love this new AI technology!" is overwhelmingly **positive**.\n\nHere\'s a breakdown of why:\n\n*   **"love"**: This is a very strong positive emotion word. It indicates deep affection, enjoyment, and approval.\n*   **"this new AI technology"**: This phrase identifies the object of the positive sentiment. The use of "new" can sometimes imply excitement or anticipation, which further contributes to the positive tone.\n\nTherefore, the sentiment is clearly **positive**.'}
